
Sacado de algún repositorio de [github](www.github.com/). Editado por [Daniel Neira](https://github.com/DanielNeira/).

Tenga en cuenta que las listas son pasadas por referencia en Python. Puede copiarlos por  `copy = original[:]`

Queremos agradecer a [Gerhard Reinelt](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/) por proporcionarnos las instancias de TSP que se pueden encontrar [aquí](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/).

--------

# Metaheuristicas: Local Search

En esta tarea vas a implementar paso a paso una estrategia de búsqueda local. 
Al final de la asignación tendrás tu propia implementación de búsqueda local iterada! 
La estrategia de búsqueda local va a ser utilizada para resolver las instancias de los vendedores viajeros en una región euclidiana. 

Para utilizar la búsqueda local se necesitan los siguientes componentes:

0. La solución codificada;
1. Una forma de crear una solución inicial;
2. La definición de barrio (`neighborhood`), es decir, los movimientos que se pueden hacer;
3. Una función objetiva;
4. Una estrategia de búsqueda.

En esta asignación, se dan la codificación y la función de objetivo. Las soluciones se codifican como una lista ordenada de símbolos de ciudad y la función objetiva es simplemente la suma de las distancias entre las ciudades.


**La siguiente celda es el código de la `caldera`, que tienes que ejecutar antes de continuar.!**

In [1]:
import utils
import importlib
import math
import random

importlib.reload(utils)

<module 'utils' from '/Users/Daniel/Downloads/HomeworkMetaheuristicsII-master/utils.py'>

---
Puede elegir entre utilizar una instancia de TSP aleatoria o una instancia de TSP suministrada. Si elige la instancia suministrada, también le daremos el valor óptimo, para que pueda utilizarla y ver cómo funciona tu solución.

In [2]:
# Corre para generar un problema aleatorio (desde la instancia berlin 52)
n, distances = utils.generate_euclid_tsp_problem(10, 0, 100)

In [3]:
# Run this to use the supplied problem.
n, distances, optimal_obj_value = utils.load_tsp_problem()

--------

## Ejercicio 1: Generación del valor inicial

Como se describe en la introducción, se requiere un valor inicial para ejecutar el algoritmo de búsqueda local. En este ejercicio se va a implementar una función que la crea.


A continuación se presenta una implementación inicial que crea soluciones aleatorias. La función `utils.objective_tsp(your_solution, distances)` calcula el valor objetivo de `r_solution`(tu solución). 

In [4]:
r_initial = utils.random_initial_value(n, distances)

print(r_initial)
print(utils.objective_tsp(r_initial, distances))

[13, 25, 29, 8, 7, 1, 10, 0, 35, 6, 30, 22, 45, 47, 38, 4, 49, 44, 26, 3, 5, 20, 39, 41, 28, 33, 2, 19, 24, 17, 27, 36, 34, 11, 15, 40, 21, 32, 18, 43, 31, 12, 14, 42, 37, 16, 51, 48, 23, 9, 50, 46]
29479.831842816267


**a. Implementar otro método para generar un valor inicial. **

In [5]:
def my_initial_value(n, distances_matrix):
    """
    Genera el valor inicial.
    
    Returns:
        Una lista ordenada de id's de ciudades.
    """
    
    # Editar esta función.
    return utils.random_initial_value(n, distances_matrix)


initial = my_initial_value(n, distances)

print(f"Valor inicial: {initial}")
print(f"Con valor objetivo: {utils.objective_tsp(initial, distances)}")

Initial value: [37, 41, 25, 20, 3, 19, 4, 36, 44, 2, 16, 35, 48, 0, 17, 27, 47, 30, 10, 13, 23, 24, 6, 29, 49, 32, 22, 51, 8, 40, 18, 38, 11, 1, 21, 28, 42, 26, 45, 43, 7, 33, 39, 14, 15, 31, 9, 12, 50, 34, 5, 46]
With objective value: 29054.042945811092


---
## Ejercicio 2: Vecindario

Ahora que tenemos la solución inicial, tenemos que ser capaces de avanzar hacia soluciones mejores. Para ello, es necesario definir el barrio.

**a. Implementar un generador de vecindario.

Sugerencia: **Un vecindario por lo general no tiene que recalcular completamente la función objetivo *¿Puede hacer esto en su caso?

Nota:** `yield` es una palabra clave usada en python para [generadores](https://wiki.python.org/moin/Generators). Puedes rendirte varias veces: ¡cada vez que "rindas" es una *solución* en el vecindario!


In [7]:
def neighbourhood(current_solution, objective_value):
    """
    Un generador que define el vecindario de `current_solution`. 
    
    Returns:
        Un generador para la tupla (solution, objective_value)
    """   

    #La implementación actual sólo devuelve la solución actual. 
    #Sugerencia: El valor_objetivo puede ser recalculado en un tiempo menor a O(n), 
    #sin embargo, puede volver a ejecutar la función utils.objective_tsp (O(n)) 
    #en cada solución en su nuevo vecindario.
    
    yield current_solution, objective_value

print(f"La solución original {initial} tiene un vecindario:")
for j in neighbourhood(initial, utils.objective_tsp(initial, distances)):
    print(f"\t{j[0]}\t Con valor objetivo: {j[1]}")


The original solution [37, 41, 25, 20, 3, 19, 4, 36, 44, 2, 16, 35, 48, 0, 17, 27, 47, 30, 10, 13, 23, 24, 6, 29, 49, 32, 22, 51, 8, 40, 18, 38, 11, 1, 21, 28, 42, 26, 45, 43, 7, 33, 39, 14, 15, 31, 9, 12, 50, 34, 5, 46] has neighboorhood:
	[37, 41, 25, 20, 3, 19, 4, 36, 44, 2, 16, 35, 48, 0, 17, 27, 47, 30, 10, 13, 23, 24, 6, 29, 49, 32, 22, 51, 8, 40, 18, 38, 11, 1, 21, 28, 42, 26, 45, 43, 7, 33, 39, 14, 15, 31, 9, 12, 50, 34, 5, 46]	 with objective value: 29054.042945811092


---
## Ejercicio 3: Búsqueda local

Ahora tenemos todos los ingredientes necesarios para realizar una simple búsqueda local. Ya lo hemos implementado para tí, sólo tiene que ejecutarlo.

In [9]:
def local_search(current_solution, distances):
    """
    Realiza una búsqueda local utilizando la solución actual como punto de partida.
    
    Returns:
       Una (solution, objective_value)
    """
    
    c_solution = current_solution
    c_objective = utils.objective_tsp(c_solution, distances)
    changed = True

    while changed:
        changed = False
        for neighbour_sequence, neighbour_objective in neighbourhood(c_solution, c_objective):
            if(neighbour_objective < c_objective):
                c_solution = neighbour_sequence
                c_objective = neighbour_objective
                changed = True
    
    return c_solution, c_objective

initial = my_initial_value(n, distances)
solution, objective = local_search(initial, distances)
print(f"Tu solución de local search es: {solution}")
print(f"Con valor objetivo: {objective}")

Your local search solution is: [35, 12, 2, 9, 18, 8, 21, 22, 27, 7, 40, 16, 48, 10, 17, 19, 13, 20, 6, 11, 34, 29, 4, 41, 49, 3, 31, 38, 24, 50, 30, 0, 32, 37, 44, 1, 33, 45, 36, 25, 51, 5, 23, 15, 39, 28, 42, 14, 46, 43, 26, 47]
With objective value: 29291.20231361705


---

# Ejercicio 4: Búsqueda local iterada 

Ahora que tenemos nuestro subprocedimiento de búsqueda local, ahora te toca a tí, tienes que implementar una búsqueda local iterada simple.

**Implementar una función de mutación, y elegir/implementar un criterio de parada.**

In [10]:
def mutate(solution):
    """
    Muta la solución dada. 
    
    Returns:
        Una solución mutada.
    """
    
    mutated = solution[:]
    # Muta la solución!
    return mutated

def iterated_local_search(n, distances):
    """
    Performs an iterated local search to find a solution for the given TSP instance.
    
    Returns:
        An optimized solution for the TSP instance.
    """
    
    initial_value = my_initial_value(n, distances)
    current_solution, c_obj = local_search(initial_value, distances)
    
    while /* TU CRITERIO DE DETENCI´*/:
        mutated_solution = mutate(current_solution)
        proposed_solution, p_obj = local_search(mutated_solution, distances)
        if p_obj < c_obj:
            current_solution = proposed_solution
            c_obj = p_obj                
    
    return current_solution, c_obj

solution, obj = iterated_local_search(n, distances)
print(f"Tu solución final es: {solution}")
print(f"Con valor objetivo: {obj}")

SyntaxError: invalid syntax (<ipython-input-10-e215d5b7ad6d>, line 24)

**¿La implementación anterior que complejidad O() tiene?¿Cómo se puede mejorar?¿Prueba ahora viendo cuando tiempo demorá tu implementación?**

## Hasta la PROXIMA!!Ç